In [1]:
import pandas as pd

which_wiki =4

In [2]:
import json

# original_sentences
with open(f'new_wiki_original_sentences_{which_wiki}.txt') as f:
    original_sentences = f.readlines()
    for i in range(len(original_sentences)):
        original_sentences[i] = original_sentences[i].strip()

# davinci-003
with open(f'../../gpt_3/gpt-outputs/zero-shot-003-batch-{which_wiki}.json') as f:
    gpt3 = json.load(f)

# turbo
with open(f'../../gpt_3/gpt-outputs/zero-shot-turbo-batch-{which_wiki}.json') as f:
    turbo = json.load(f)

# gpt-4
with open(f"../../gpt_3/gpt-outputs/zero-shot-gpt-4-batch-{which_wiki}.json") as f:
    gpt_4 = json.load(f)

# alpaca-7b
with open(f'alpaca-7B-new-wiki-{which_wiki}-output.txt') as f:
    alpaca_7b = f.readlines()
    for i in range(len(alpaca_7b)):
        alpaca_7b[i] = alpaca_7b[i].strip()

# vicuna-7b
with open(f'vicuna-7B-new-wiki-{which_wiki}-output.txt') as f:
    vicuna_7b = f.readlines()
    for i in range(len(vicuna_7b)):
        vicuna_7b[i] = vicuna_7b[i].strip()

In [3]:
# make dataframe with all outputs
df = pd.DataFrame()
df['Original Sentence'] = original_sentences
df['Davinci-003'] = gpt3
df['Turbo'] = turbo
df['GPT-4'] = gpt_4
df['Alpaca-7b'] = alpaca_7b
df['Vicuna-7b'] = vicuna_7b

In [4]:
df.to_csv(f'all-outputs-new-wiki-{which_wiki}-before-splitting.csv', index=False)

In [5]:
from string import punctuation
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

def remove_tokenization_artifacts(s, src):
    stokens = s.split()
    snew = s
    for i, token in enumerate(stokens):
        if i > 0 and i < len(stokens) - 1 and token in punctuation:
            substrboth = stokens[i - 1] + token + stokens[i + 1]
            substrleft = stokens[i - 1] + token
            substright = token + stokens[i + 1]
            if substrboth in src:
                snew = snew.replace(stokens[i - 1] + " " + token + " " + stokens[i + 1], substrboth)
            elif substrleft in src:
                snew = snew.replace(stokens[i - 1] + " " + token, substrleft)
            elif substright in src:
                snew = snew.replace(token + " " + stokens[i + 1], substright)

    snew = snew.replace("''", '"')
    snew = snew.replace(" .", ".")
    snew_rest = "" if len(snew) == 1 else snew[1:]
    if len(snew) > 0:
        snew = snew[0].capitalize() + snew_rest
    snew = snew.replace("-lrb-", "(").replace("-rrb-", ")")
    snew = snew.replace("-LRB-", "(").replace("-RRB-", ")")
    return snew

def reformat_output(output, original):
    output = remove_tokenization_artifacts(output, original)
    output = " || ".join(sent_tokenize(output))
    return output

[nltk_data] Downloading package punkt to /Users/yaod_1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# reformat output with reformat_output function
for index, row in df.iterrows():
    df.loc[index, 'Davinci-003'] = reformat_output(row['Davinci-003'], row['Original Sentence'])
    df.loc[index, 'Turbo'] = reformat_output(row['Turbo'], row['Original Sentence'])
    df.loc[index, 'GPT-4'] = reformat_output(row['GPT-4'], row['Original Sentence'])
    df.loc[index, 'Alpaca-7b'] = reformat_output(row['Alpaca-7b'], row['Original Sentence'])
    df.loc[index, 'Vicuna-7b'] = reformat_output(row['Vicuna-7b'], row['Original Sentence'])

In [7]:
df.to_csv(f'all-outputs-new-wiki-{which_wiki}.csv', index=False)